<a href="https://colab.research.google.com/github/obscraft23/SlimeChunkFinderAddonBE/blob/master/chatgpt_wrapper_mcbe_demo_jp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**TL;DL**

このnotebookはminecraft統合版からwebsocketでchatgptを使うためのサーバーを走らせるためのものです。以下の指示に従って操作するだけで、特別な環境を用意しなくても、簡単にマイクラ内からchatgptが使えるようになります。

最終更新: 2023-02-24

**注意事項（必読）**

+ このノートブックの使用は使用者の自己責任で行ってください。作成者のobscraft23は一切の責任を負いません。

+ chatgptのアカウントを入力する必要があります。このノートブック自体にそうした情報を収集するようなプログラムは当然含まれていませんが、chatgptに接続するにあたり、third-party製のreverse proxyサーバーを経由します。この点に留意してください。詳しい情報は https://github.com/transitive-bullshit/chatgpt-api#reverse-proxy を参照。
心配な人はchatgptの有料アカウントではなく、無料アカウントを使用することでリスクを抑えられます。

+ chatgptは2023年2月現在preview段階で、仕様も日・週単位で変わることがあります。このノートブックも恒久的に使用できるとは限りません。可能であれば以下のgithubリポジトリを通してupdateします。
<br>https://github.com/obscraft23/chatgpt-mcbe

+ [MITライセンス](https://github.com/obscraft23/chatgpt-mcbe/blob/master/LICENSE)を適用します。クレジット表記を行えば、複製・再配布・改変等は自由です。



**必要なもの**
+ **マインクラフト統合版**
<br>バージョン依存性は特にないと思いますが、動作確認は1.19.62で行っています。
<br>スマホ単体でも動きます（iOSで動作テストしました）。
<br>/connect コマンドを使うのでBDSは不可です。

+ **googleアカウント**
<br>このnotebookを実行するのに必要です。googleにログインせずにこのnotebookを開いた場合、はgoogleアカウントにログイン後に再度開きなおしてください。

+ **chatgptのアカウント**
<br>chatgptのアカウントをまだ持っていない人は以下のサイトから"Sing up"にアクセスして無料アカウントを作成してください。
<br>https://chat.openai.com/auth/login
<br>(注意)必ずメールアドレスから新規作成してください。**Google及びMicrosoftアカウント連携は不可です**。
<br>SMS認証が求められるので携帯電話番号が必要です。




**アカウント情報の入力**

以下にchatgptの登録メールアドレスとパスワードを入力してください。
繰り返しになりますが、入力は自己責任です。

登録メールアドレス

In [ ]:
input_email_address = "example@gmail.com" #@param {type:"string"}

パスワード

In [ ]:
input_password = "examplepassward" #@param {type:"string"}

### **サーバーの起動**

ここまで完了したら、上のメニューから
「ランタイム」＞「すべてのセルを実行」を押してください。

「警告: このノートブックは Google が作成したものではありません。」という警告が出た場合は「このまま実行」を押してください。

エラーが出ずに「start server」下に以下のような文が表示されたら、準備完了です。

```
サーバーを起動したら以下のコマンドをマイクラ内のチャットに入力してください。
（コマンド）

-- log --
Ready 0.0.0.0:19132
```

マイクラを起動して指示されたコマンドをチャットに入力してください。


In [ ]:
#@title debug (you can ignore)
debug = False
if debug:
  import logging
  requests_log = logging.getLogger("urllib3")
  requests_log.setLevel(logging.DEBUG)
  import http
  http.client.HTTPConnection.debuglevel=1

In [ ]:
#@title additional packages
import os
if not os.path.exists("ngrok"):
  cmdres = os.system("wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip")
  cmdres = os.system("unzip ngrok-stable-linux-amd64.zip")

cmdres = os.system("pip install websockets")
cmdres = os.system("pip install nest_asyncio")
cmdres = os.system("pip install git+https://github.com/acheong08/OpenAIAuth.git")

In [ ]:
#@title get access token
import OpenAIAuth
myobj = OpenAIAuth.Authenticator(email_address=input_email_address,password=input_password)
myobj.begin()
access_token = myobj.get_access_token()

In [ ]:
#@title websocket server object
import asyncio
import sys
import json
import uuid
import websockets
from websockets import serve
import nest_asyncio
nest_asyncio.apply()

class WsClient:
  def start(self,host="0.0.0.0", port=19132):
    self.host = host
    self.port = port
    self.ws = websockets.serve(self.receive, self.host, self.port)
    self.loop = asyncio.get_event_loop()
    self.loop.run_until_complete(self.ws)
    self.event_ready()
    self.loop.run_forever()

  async def receive(self, websocket, path):
    self.ws = websocket
    await self.listen_event()
    await self.event("connect")  # self.event_connect()
    try:
      while True:
        data = await self.ws.recv()
        msg = json.loads(data)
        await self.parse_command(msg)
    except (
        websockets.exceptions.ConnectionClosedOK,
        websockets.exceptions.ConnectionClosedError,
        websockets.exceptions.ConnectionClosed):
      await self.event("disconnect")  # self.event_disconnect()
      sys.exit()

  async def listen_event(self):
    for event in self.events:
      await self.ws.send(json.dumps({
        "body": {"eventName": event},
        "header": {
          "requestId": "00000000-0000-0000-0000-000000000000",
          "messagePurpose": "subscribe",
          "version": 1,
          "messageType": "commandRequest"}
        }))
      
  async def parse_command(self, message):
    if message["header"]["messagePurpose"] == "event":
      event_name = message["header"]["eventName"]
      await self.event(event_name, message)
      if message["header"]["eventName"] == "PlayerMessage" and message["body"]["type"] == 'chat':
        pass
    elif message["header"]["messagePurpose"] == "error":
      await self.event("error", message)

  async def command(self, cmd):
    uuid4 = str(uuid.uuid4())
    cmd_json = json.dumps({
      "body": {"origin": {"type": "player"},"commandLine": cmd,"version": 1},
      "header": {"requestId": uuid4,"messagePurpose": "commandRequest","version": 1,"messageType": "commandRequest"}
      })
    
    await self.ws.send(cmd_json)
    data = await self.ws.recv()
    msg = json.loads(data)
    if msg["header"]["messagePurpose"] == "commandResponse" and msg["header"]["requestId"] == uuid4:
      return msg
    else:
      return None

  async def event(self, name, *args):
    func = f"self.event_{name}"
    
    if args == ():
      try:
        await eval(f"{func}()")
      except NameError as e:
        print(f"event_{name}")
        
    else:
      try:
        await eval(f"{func}({args})")
      except NameError as e:
        print(f"event_{name}")

  ###
  async def event_connect(self):
    mcresptext = "§a[Server]§r welcome!"
    await self.command('tellraw @a {"rawtext":[{"text":"'+mcresptext+'"}]} ')
    print("Connected!")

  async def event_disconnect(self):
    print("disconnect!")

  def event_ready(self):
    print(f"Ready {self.host}:{self.port}")

    self.events = ["PlayerMessage", "PlayerDied"]

  async def event_PlayerMessage(self, event):
    rawtext = event[0]['body']['message']
    print("[Input text] "+rawtext)
    resptext = getChatgpt(rawtext)
    print("[ChatGPT] "+resptext)
    mcresptext = "§d[ChatGPT]§r "+resptext
    await self.command('tellraw @a {"rawtext":[{"text":"'+mcresptext+'"}]} ')

  async def event_PlayerDied(self, event):
    pass
    #print(event)

In [ ]:
#@title getChatgpt function
def getChatgpt(inputtext):

  import uuid
  import requests
  import json
  import os
  import re
  #import time

  if os.path.exists('.mcservercache/cid.log'):
    with open('.mcservercache/cid.log','r') as f:
      cid = f.read()
  
  else:
    cid = None
  
  if os.path.exists('.mcservercache/mid.log'):
    with open('.mcservercache/mid.log','r') as f:
      mid = f.read()
  
  else:
    mid = None

  headers = {
    'Authorization': 'Bearer {}'.format(access_token),
    'Accept': 'text/event-stream',
    'Content-Type': "application/json",
    "Referer": "https://chat.openai.com/chat",
    "Origin": "https://chat.openai.com",
    "User-Agent": 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15',
    "OpenAI-Client-Id": 'chat'}
  
  if mid != None and cid != None:
    payload = {
      "action":"next",
      "conversation_id":cid,
      "messages":[{"id":str(uuid.uuid4()),
                  "role": 'user',
                  "content": {"content_type": 'text',"parts": [inputtext]}
                  }],
      "model": "text-davinci-002-render-sha",
      "parent_message_id":mid,
    }
    
  else:
    payload = {
      "action":"next",
      "messages":[{"id":str(uuid.uuid4()),
                  "role": 'user',
                  "content": {"content_type": 'text',"parts": [inputtext]}
                    }],
      "model": "text-davinci-002-render-sha",
      "parent_message_id":str(uuid.uuid4()),
    }
  
  url = "https://chatgpt.duti.tech/api/conversation"

  res = requests.post(url, data=json.dumps(payload), headers=headers)

  try:
    response_text = res.text.replace("data: [DONE]", "")
    data = re.findall(r'data: (.*)', response_text)[-1]
    as_json = json.loads(data)
    cid =  as_json['conversation_id']
    mid = as_json['message']['id']
    outputttest = as_json['message']['content']['parts'][0]

  except:
    outputttest = json.loads(res.text)['detail']

  if not os.path.exists('./.mcservercache'):
    os.mkdir('./.mcservercache')

  if cid!=None:
    with open('.mcservercache/cid.log','w') as f:
      f.write(cid)
  
  if mid!=None:
    with open('.mcservercache/mid.log','w') as f:
      f.write(mid)
  
  return outputttest

In [ ]:
#@title establish ngrok
import time
get_ipython().system_raw('./ngrok http 19132 &')
time.sleep(5)

### サーバー

In [ ]:
#@title start server
import requests
import json
res = requests.get("http://localhost:4040/api/tunnels")
json_str = res.content.decode("utf-8")
json_dict = json.loads(json_str)
wsaddress = json_dict['tunnels'][0]['public_url'].replace('https://','ws://').replace('http://','ws://')
print("サーバーを起動したら以下のコマンドをマイクラ内のチャットに入力してください。")
print("/connect "+wsaddress)
print("")
print("-- log --")
myobj = WsClient()
myobj.start(host="0.0.0.0",port=19132)

**マイクラ内の操作**

コマンドをマイクラ内から実行し、

[Server] welcome!

と応答が帰ってきたら、接続成功です。以後チャットに入力した文章は、chatgptに送信され、chatgptと対話できるようになります。

**注意**

+ chatgptからの応答は時間がかかる場合や返答自体が返ってこない場合があります。入力した文章の内容やchatgpt全体の使用状況に依存します。

+ 短い時間にたくさんの対話を行うと無料アカウントの場合、一定時間使用できなくなる場合があります。

+ 返答が長い場合途中で終わってしまうことがあります。その場合は「続けて」などと入力することで、続きを聞くことができます。

+ 何もしなければ、以前の会話状態を記憶したまま対話が進行します。会話をリセットしたい場合は以下のセルを実行してください。（一度サーバーを切断する必要があります）

In [ ]:
#@title 会話のリセット
%rm -rf .mcservercache

**終了・再起動**

+ サーバーを切断するには上のメニューから 「ランタイム」＞「ランタイムを再起動」を押してください。

+ サーバーを再起動する場合は、もう一度上のメニューから 「ランタイム」＞「すべてのセルを実行」を押してください。**websocketサーバーアドレスは毎回変わるので注意してください。**

+ このノートブックの使用をやめる場合は、「ランタイム」＞「ランタイムを接続解除してから削除」を押してから、ページを離れてください。

**トラブルシューティング**

+ よくあるエラー原因
  - chatgptのアカウント情報が間違っている
  - chatgpt側でエラーが起きている（No conversation foundなど）

+ 何かトラブルやエラーが発生した場合は、「ランタイム」＞「ランタイムを再起動」または「ランタイム」＞「ランタイムを接続解除してから削除」をしてから、再度「ランタイム」＞「すべてのセルを実行」を押してみると解決することがあります。

+ chatgpt側で問題が起きている（短時間にたくさん会話するなど）場合は、時間を空けて試す以外解決方法がありません。